In [20]:
# Tải dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hoantainson/dataset-weather-vit-nam-trong-1-nm-li")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\ADMIN\.cache\kagglehub\datasets\hoantainson\dataset-weather-vit-nam-trong-1-nm-li\versions\24


In [21]:
# Thư viện cần dùng
import pandas as pd
import os

## 2.1. Data Collection

### 1. What subject is your data about?
* **Topic & Domain:** Dữ liệu thời tiết hàng ngày (Daily Weather Data) tại các tỉnh thành của Việt Nam.
* **Real-world context:** Bộ dữ liệu phản ánh điều kiện khí tượng thực tế (nhiệt độ, độ ẩm, lượng mưa, tốc độ gió, chỉ số UV, v.v.) tại 63 tỉnh thành trên khắp Việt Nam. Thông tin này đóng vai trò quan trọng trong việc dự báo thời tiết, lên kế hoạch nông nghiệp, du lịch và các hoạt động đời sống hàng ngày.

### 2. What is the source of your data?
* **Platform:** Kaggle.
* **URL:** [Dataset Weather Viet Nam trong 1 nam lai](https://www.kaggle.com/datasets/hoantainson/dataset-weather-vit-nam-trong-1-nm-li)
* **Author:** Hoàng Thanh Sơn - Sinh viên năm 3 tại Đại học Công nghệ Thông tin (UIT) (Kaggle user: `hoantainson`).
* **Publication Date:** Được cập nhật khoảng 6 tháng trước (dựa trên metadata hiện tại).
* **Collection Date:** Từ `2024-04-21` đến `2025-06-04`.

### 3. Is this data licensed for your use?
* **License:** MIT License.
* **Educational Use Permission:** **Có**. Giấy phép MIT là mã nguồn mở, cho phép sử dụng, sao chép, sửa đổi và phân phối miễn phí, hoàn toàn phù hợp cho mục đích giáo dục và nghiên cứu.

> **MIT License Detail:**
>
> Copyright (c) 2013 Mark Otto.  
> Copyright (c) 2017 Andrew Fong.
>
> Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software...
>
> *THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND...*

### 4. How was this data collected?
* **Collection Method:** Gọi API từ [weatherapi.com](https://www.weatherapi.com/) – nguồn dữ liệu thời tiết lịch sử và dự báo uy tín.
* **Quy trình xử lý:**
    1.  **Retrieval:** Dùng API truy xuất dữ liệu thô (nhiệt độ, lượng mưa...) cho từng tỉnh.
    2.  **Exploratory Data Analysis (EDA):** Kiểm tra cấu trúc, tìm mẫu hình (patterns) và phát hiện điểm bất thường/giá trị thiếu.
    3.  **Data Cleaning:** Xử lý sự không nhất quán để đảm bảo độ chính xác.
    4.  **Compilation:** Tổng hợp dữ liệu sạch thành định dạng `.csv`.
* **Target Population:** 63 tỉnh/thành phố tại Việt Nam.
* **Time Period:** 410 ngày (Từ `2024-04-21` đến `2025-06-04`).
* **Limitations:** Do tổng hợp từ bên thứ ba (API), có thể tồn tại sai số đo đạc từ trạm gốc hoặc giá trị bị thiếu (missing values) khi request.

### 5. Why did you choose this dataset?
* **Interest (Lý do chọn đề tài):**
    * Tính cấp thiết: Việt Nam đang đối mặt với thời tiết cực đoan (bão, lũ) gây thiệt hại lớn về người và tài sản.
    * Mục tiêu: Phân tích dữ liệu 63 tỉnh thành giúp hiểu quy luật khí tượng, hỗ trợ cảnh báo sớm và giảm nhẹ thiên tai.
* **Potential Insights:** Bộ dữ liệu đủ phức tạp để thực hiện các mô hình Machine Learning:
    * **Regression:** Dự báo nhiệt độ trung bình ngày hôm sau dựa trên dữ liệu lịch sử.
    * **Classification:** Dự đoán khả năng có mưa (`day.daily_will_it_rain`) dựa trên độ ẩm và áp suất.

In [22]:
csv_file = os.path.join(path, "df_weather.csv")

# Đọc dữ liệu
df = pd.read_csv(csv_file)

column_groups = {
    
    "📍 Thông tin Địa lý": [
        "location.name",      # ✅ GIỮ - Tên tỉnh/thành phố
        "location.region",    # ✅ GIỮ - Vùng miền (quan trọng cho phân tích)
        "location.terrain",   # ✅ GIỮ - Địa hình (đồng bằng, miền núi, ven biển)
        "location.country",   # ⚠️  XÓA - Tất cả đều là Vietnam
        "location.lat",       # ✅ GIỮ - Vĩ độ (dùng cho visualization map)
        "location.lon",       # ✅ GIỮ - Kinh độ (dùng cho visualization map)
    ],
    
    "📅 Thông tin Thời gian": [
        "date",              # ✅ GIỮ + CHUYỂN ĐỔI sang datetime
        "date_epoch",        # ⚠️  XÓA - Dư thừa (có thể tạo lại từ date)
    ],
    
    "🌡️  Nhiệt độ (Temperature)": [
        "day.maxtemp_c",     # ✅ GIỮ - Nhiệt độ max (Celsius)
        "day.mintemp_c",     # ✅ GIỮ - Nhiệt độ min (Celsius)
        "day.avgtemp_c",     # ✅ GIỮ - Nhiệt độ trung bình (Celsius)
        "day.maxtemp_f",     # ⚠️  XÓA - Dư thừa (Fahrenheit)
        "day.mintemp_f",     # ⚠️  XÓA - Dư thừa
        "day.avgtemp_f",     # ⚠️  XÓA - Dư thừa
    ],
    
    "💨 Gió & Mưa (Wind & Precipitation)": [
        "day.maxwind_kph",     # ✅ GIỮ - Tốc độ gió max (km/h)
        "day.maxwind_mph",     # ⚠️  XÓA - Dư thừa (mph)
        "day.totalprecip_mm",  # ✅ GIỮ - Lượng mưa (mm)
        "day.totalprecip_in",  # ⚠️  XÓA - Dư thừa (inches)
        "day.totalsnow_cm",    # ⚠️  XÓA - Việt Nam không có tuyết
    ],
    
    "👁️  Tầm nhìn & Độ ẩm (Visibility & Humidity)": [
        "day.avgvis_km",       # ✅ GIỮ - Tầm nhìn (km)
        "day.avgvis_miles",    # ⚠️  XÓA - Dư thừa (miles)
        "day.avghumidity",     # ✅ GIỮ - Độ ẩm %
    ],
    
    "☀️ Điều kiện thời tiết (Weather Conditions)": [
        "day.daily_will_it_rain",      # ✅ GIỮ + CHUYỂN ĐỔI sang boolean
        "day.daily_chance_of_rain",    # ✅ GIỮ - Xác suất mưa %
        "day.daily_will_it_snow",      # ⚠️  XÓA - Không liên quan VN
        "day.daily_chance_of_snow",    # ⚠️  XÓA - Không liên quan VN
        "day.condition.text",          # ✅ GIỮ - Mô tả thời tiết
        "day.condition.icon",          # ⚠️  XÓA - Link icon, không cần thiết
        "day.condition.code",          # ✅ GIỮ - Mã điều kiện (có thể dùng)
        "day.uv",                      # ✅ GIỮ - Chỉ số UV
    ],
    
    "🌙 Dữ liệu Thiên văn (Astronomical)": [
        "astro.sunrise",        # ✅ GIỮ (nếu cần) - Thời gian mặt trời mọc
        "astro.sunset",         # ✅ GIỮ (nếu cần) - Thời gian mặt trời lặn
        "astro.moonrise",       # ⚠️  CÂN NHẮC - Ít dùng trong phân tích
        "astro.moonset",        # ⚠️  CÂN NHẮC - Ít dùng trong phân tích
        "astro.moon_phase",     # ⚠️  CÂN NHẮC - Ít dùng trong phân tích
        "astro.moon_illumination", # ⚠️  CÂN NHẮC - Ít dùng trong phân tích
    ],
}

BƯỚC 1: XÓA CÁC CỘT DƯ THỪA
✂️  Xóa các cột:
   - location.country (tất cả là Vietnam)
   - date_epoch (tính lại được từ date)
   - Các cột Fahrenheit: maxtemp_f, mintemp_f, avgtemp_f
   - Các cột mph/miles/inches: maxwind_mph, avgvis_miles, totalprecip_in
   - Các cột về tuyết: totalsnow_cm, daily_will_it_snow, daily_chance_of_snow
   - day.condition.icon (link không cần thiết)
   - (Tùy chọn) Các cột thiên văn về mặt trăng nếu không dùng

BƯỚC 2: CHUYỂN ĐỔI KIỂU DỮ LIỆU
🔄 Chuyển đổi:
   - date → datetime64 (dùng pd.to_datetime)
   - location.name → category
   - location.region → category
   - location.terrain → category
   - day.condition.text → category
   - day.daily_will_it_rain → boolean (0→False, 1→True)

BƯỚC 3: XỬ LÝ DỮ LIỆU BẤT THƯỜNG
🔧 Kiểm tra và sửa:
   - Xóa/sửa hàng có maxtemp_c < mintemp_c
   - Kiểm tra nhiệt độ âm (không hợp lý ở VN)
   - Kiểm tra độ ẩm ngoài 0-100%
   - Xử lý giá trị âm trong lượng mưa (nếu có)

BƯỚC 4: TẠO CÁC CỘT MỚI (FEATURE ENGINEERING)
➕ Tạo thêm:
   - temperature_range = maxtemp_c - mintemp_c (biên độ nhiệt)
   - month, quarter, season từ cột date
   - is_rainy = totalprecip_mm > 0 (ngày có mưa)
   - rain_category = phân loại mưa (nhỏ/vừa/to)
   - temp_category = phân loại nhiệt độ (lạnh/mát/nóng)

## 2.2. Data Exploration

In [23]:
print("DATASET OVERVIEW\n")
print("Basic Information:")

num_rows = len(df)
num_cols = len(df.columns)

memory_usage_bytes = df.memory_usage(deep=True).sum()
memory_usage_mb = memory_usage_bytes / (1024 * 1024)

print(f"The dataset has {num_rows:,} rows.")
print(f"The dataset has {num_cols} columns.")
print(f"Each row represents the daily weather summary for a specific location.")
print(f"The overall size of the dataset is {memory_usage_mb:.2f} MB.")

print("\nData Integrity: ")

# 1. Check for duplicates
duplicate_count = df.duplicated().sum()

if duplicate_count > 0:
    print(f"Yes, there are {duplicate_count} duplicated rows.")
else:
    print("No, there are no duplicated rows.")

# 2. Check for empty rows
empty_rows_count = df.isnull().all(axis=1).sum()

if empty_rows_count > 0:
    print(f"No, there are {empty_rows_count} entirely empty rows.")
else:
    print("Yes, all rows are complete (no entirely empty rows).")

# Remove duplicates
if duplicate_count > 0:
    print(f"\nRemoving {duplicate_count} duplicated rows...")
    df = df.drop_duplicates()
    print(f"Duplicates removed")


DATASET OVERVIEW

Basic Information:
The dataset has 26,018 rows.
The dataset has 34 columns.
Each row represents the daily weather summary for a specific location.
The overall size of the dataset is 25.31 MB.

Data Integrity: 
Yes, there are 252 duplicated rows.
Yes, all rows are complete (no entirely empty rows).

Removing 252 duplicated rows...
Duplicates removed


**Decision: Duplicates should be removed**

**Justification:**

1. **Data Accuracy**: In this weather dataset, each row corresponds to the specific weather conditions of a unique location on a unique date. Having duplicates implies redundant or erroneous data collection.

2. **Statistical Validity**: Keeping duplicates would artificially inflate the sample size, leading to biased statistical metrics.

3. **Model Performance**: If used for machine learning, duplicates can cause data leakage.

### 2.2. Data Exploration - Column Inventory

**1. What is the meaning/definition of each column?**

Dựa trên thông tin của bộ dữ liệu, dưới đây là định nghĩa chi tiết cho từng cột:

| **Nhóm** | **Tên Cột** | **Định nghĩa** |
| :--- | :--- | :--- |
| **Thông tin Địa điểm** | `location.name` | Tên thành phố/tỉnh (ví dụ: An Giang, Hà Nội). |
| | `location.region` | Vùng miền hành chính (ví dụ: Đồng Bằng Sông Cửu Long). |
| | `location.terrain` | Loại địa hình của địa điểm (ví dụ: đồng bằng, ven biển, miền núi). |
| | `location.country` | Tên quốc gia (Vietnam). |
| | `location.lat` | Tọa độ Vĩ độ của địa điểm. |
| | `location.lon` | Tọa độ Kinh độ của địa điểm. |
| **Thông tin Thời gian** | `date` | Ngày quan sát (định dạng YYYY-MM-DD). |
| | `date_epoch` | Ngày quan sát dưới định dạng Unix epoch. |
| **Nhiệt độ** | `day.maxtemp_c` | Nhiệt độ cao nhất trong ngày (độ C). |
| | `day.maxtemp_f` | Nhiệt độ cao nhất trong ngày (độ F). |
| | `day.mintemp_c` | Nhiệt độ thấp nhất trong ngày (độ C). |
| | `day.mintemp_f` | Nhiệt độ thấp nhất trong ngày (độ F). |
| | `day.avgtemp_c` | Nhiệt độ trung bình trong ngày (độ C). |
| | `day.avgtemp_f` | Nhiệt độ trung bình trong ngày (độ F). |
| **Gió & Lượng mưa** | `day.maxwind_kph` | Tốc độ gió tối đa tính bằng km/h. |
| | `day.maxwind_mph` | Tốc độ gió tối đa tính bằng dặm/h (mph). |
| | `day.totalprecip_mm` | Tổng lượng mưa tính bằng milimet (mm). |
| | `day.totalprecip_in` | Tổng lượng mưa tính bằng inch. |
| | `day.totalsnow_cm` | Tổng lượng tuyết rơi tính bằng centimet (cm). |
| **Tầm nhìn & Độ ẩm** | `day.avgvis_km` | Tầm nhìn xa trung bình tính bằng km. |
| | `day.avgvis_miles` | Tầm nhìn xa trung bình tính bằng dặm (miles). |
| | `day.avghumidity` | Độ ẩm trung bình (%). |
| **Điều kiện Thời tiết** | `day.condition.text` | Mô tả thời tiết bằng văn bản (ví dụ: Nắng, Có thể có mưa rải rác). |
| | `day.condition.icon` | Đường dẫn (URL) đến hình ảnh biểu tượng thời tiết. |
| | `day.condition.code` | Mã định danh duy nhất đại diện cho điều kiện thời tiết. |
| | `day.daily_will_it_rain` | Giá trị Boolean chỉ báo sẽ có mưa hay không (1 = Có, 0 = Không). |
| | `day.daily_chance_of_rain` | Xác suất/Khả năng có mưa tính theo phần trăm (%). |
| | `day.uv` | Chỉ số tia cực tím (UV Index) hàng ngày. |
| **Dữ liệu Thiên văn** | `astro.sunrise` | Thời gian mặt trời mọc. |
| | `astro.sunset` | Thời gian mặt trời lặn. |
| | `astro.moonrise` | Thời gian trăng mọc. |
| | `astro.moonset` | Thời gian trăng lặn. |
| | `astro.moon_phase` | Chu kỳ (pha) của mặt trăng (ví dụ: Trăng khuyết đầu tháng). |
| | `astro.moon_illumination` | Tỷ lệ phần trăm độ sáng của mặt trăng. |


**2. Which columns are relevant to potential analysis?**

## 2. Những cột dữ liệu nào phù hợp cho việc phân tích tiềm năng?

Các cột sau đây đã được lựa chọn cho quá trình phân tích:

### Thông tin Địa lý
- `location.name`
- `location.region`
- `location.terrain`
- `location.lat`
- `location.lon`

### Thông tin Thời gian
- `date`

### Nhiệt độ
- `day.maxtemp_c`
- `day.mintemp_c`
- `day.avgtemp_c`

### Gió & Mưa
- `day.maxwind_kph`
- `day.totalprecip_mm`

### Tầm nhìn & Độ ẩm
- `day.avgvis_km`
- `day.avghumidity`

### Điều kiện thời tiết
- `day.daily_will_it_rain`
- `day.daily_chance_of_rain`
- `day.condition.text`
- `day.condition.code`
- `day.uv`

### Dữ liệu Thiên văn
- `astro.sunrise`
- `astro.sunset`


**3. Are there any columns that should be dropped? Why?**

Chúng tôi quyết định loại bỏ các cột sau để đảm bảo chất lượng dữ liệu, loại bỏ sự dư thừa và tập trung vào bối cảnh khí hậu Việt Nam:

* **Đơn vị đo lường dư thừa (Không phải hệ mét):**
    * `day.maxtemp_f`, `day.mintemp_f`, `day.avgtemp_f`: Việt Nam sử dụng thang đo Celsius; độ Fahrenheit là dư thừa.
    * `day.maxwind_mph`, `day.totalprecip_in`, `day.avgvis_miles`: Việt Nam sử dụng hệ mét (km/h, mm, km); các đơn vị đo lường Anh/Mỹ là không cần thiết.

* **Ngữ cảnh không phù hợp (Tuyết):**
    * `day.totalsnow_cm`: Việt Nam là quốc gia nhiệt đới. Tuyết cực kỳ hiếm gặp và không có ý nghĩa thống kê trong bộ dữ liệu này, khiến các cột này trở nên vô nghĩa.

* **Cột chỉ có một giá trị duy nhất:**
    * `location.country`: Toàn bộ dữ liệu chỉ giới hạn trong phạm vi "Vietnam", do đó cột này không có sự biến thiên để phân tích.

* **Metadata dư thừa hoặc mang tính kỹ thuật:**
    * `date_epoch`: Dư thừa vì chúng ta đã có cột `date` ở định dạng con người có thể đọc được.
    * `day.condition.icon`: Cột này chứa đường dẫn ảnh, không hữu ích cho việc phân tích thống kê hay mô hình hóa.

* **Độ liên quan thấp (Dữ liệu Thiên văn):**
    * `astro.moonrise`, `astro.moonset`, `astro.moon_phase`, `astro.moon_illumination`: Mặc dù có sẵn, nhưng chu kỳ mặt trăng có tác động trực tiếp không đáng kể đến việc dự báo thời tiết hàng ngày so với các yếu tố mặt trời và khí quyển. Các cột này được loại bỏ để giảm số chiều dữ liệu.

In [24]:
# Danh sách các cột cần xóa (dựa trên phân tích ở trên)
columns_to_drop = [
    "location.country",         # Tất cả đều là Vietnam
    "date_epoch",               # Dư thừa
    "day.maxtemp_f",            # Dư thừa (Fahrenheit)
    "day.mintemp_f",            # Dư thừa (Fahrenheit)
    "day.avgtemp_f",            # Dư thừa (Fahrenheit)
    "day.maxwind_mph",          # Dư thừa (mph)
    "day.totalprecip_in",       # Dư thừa (inches)
    "day.totalsnow_cm",         # Không liên quan VN
    "day.avgvis_miles",         # Dư thừa (miles)
    "day.condition.icon",       # Metadata không cần thiết
    "astro.moonrise",           # Ít dùng
    "astro.moonset",            # Ít dùng
    "astro.moon_phase",         # Ít dùng
    "astro.moon_illumination",  # Ít dùng
]

# Thực hiện xóa cột
# errors='ignore' giúp tránh lỗi nếu cột không tồn tại trong dataframe
df.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')

print("Đã xóa các cột không cần thiết.")
print(f"Số lượng cột còn lại: {len(df.columns)}")
print("\nDanh sách cột hiện tại:")
print(df.columns.tolist())

Đã xóa các cột không cần thiết.
Số lượng cột còn lại: 20

Danh sách cột hiện tại:
['location.name', 'location.region', 'location.terrain', 'location.lat', 'location.lon', 'date', 'day.maxtemp_c', 'day.mintemp_c', 'day.avgtemp_c', 'day.maxwind_kph', 'day.totalprecip_mm', 'day.avgvis_km', 'day.avghumidity', 'day.daily_will_it_rain', 'day.daily_chance_of_rain', 'day.condition.text', 'day.condition.code', 'day.uv', 'astro.sunrise', 'astro.sunset']


In [25]:
print("\nData Types:")
print("The current data type of each column: ")
print(df.dtypes)

# 1. Chuyển date sang datetime64
df['date'] = pd.to_datetime(df['date'])
print("Converted 'date' from object to datetime64")

# 2. Chuyển location.name sang category
df['location.name'] = df['location.name'].astype('category')
print("Converted 'location.name' from object to category")

# 3. Chuyển location.region sang category
df['location.region'] = df['location.region'].astype('category')
print("Converted 'location.region' from object to category")

# 4. Chuyển location.terrain sang category
df['location.terrain'] = df['location.terrain'].astype('category')
print("Converted 'location.terrain' from object to category")

# 5. Chuyển day.condition.text sang category
df['day.condition.text'] = df['day.condition.text'].astype('category')
print("Converted 'day.condition.text' from object to category")

# 6. Chuyển day.daily_will_it_rain sang boolean
df['day.daily_will_it_rain'] = df['day.daily_will_it_rain'].astype('bool')
print("Converted 'day.daily_will_it_rain' from int64 to bool")

df['astro.sunrise'] = pd.to_datetime(
    df['date'].astype(str) + ' ' + df['astro.sunrise'],
    format='%Y-%m-%d %I:%M %p',
    errors='coerce'
)
print("Converted 'astro.sunrise' to datetime64")

df['astro.sunset'] = pd.to_datetime(
    df['date'].astype(str) + ' ' + df['astro.sunset'],
    format='%Y-%m-%d %I:%M %p',
    errors='coerce'
)
print("Converted 'astro.sunset' to datetime64")




Data Types:
The current data type of each column: 
location.name                object
location.region              object
location.terrain             object
location.lat                float64
location.lon                float64
date                         object
day.maxtemp_c               float64
day.mintemp_c               float64
day.avgtemp_c               float64
day.maxwind_kph             float64
day.totalprecip_mm          float64
day.avgvis_km               float64
day.avghumidity               int64
day.daily_will_it_rain        int64
day.daily_chance_of_rain      int64
day.condition.text           object
day.condition.code            int64
day.uv                      float64
astro.sunrise                object
astro.sunset                 object
dtype: object
Converted 'date' from object to datetime64
Converted 'location.name' from object to category
Converted 'location.region' from object to category
Converted 'location.terrain' from object to category
Converted 'day.c

## Data Types

**1. What is the current data type of each column?**

Dựa trên tổng quan về tập dữ liệu, các kiểu dữ liệu hiện tại như sau:

| Column Name | Current Data Type |
| :--- | :--- |
| `location.name` | `object` |
| `location.region` | `object` |
| `location.terrain` | `object` |
| `location.lat` | `float64` |
| `location.lon` | `float64` |
| `date` | `object` |
| `day.maxtemp_c` | `float64` |
| `day.mintemp_c` | `float64` |
| `day.avgtemp_c` | `float64` |
| `day.maxwind_kph` | `float64` |
| `day.totalprecip_mm` | `float64` |
| `day.avgvis_km` | `float64` |
| `day.avghumidity` | `int64` |
| `day.daily_will_it_rain` | `int64` |
| `day.daily_chance_of_rain` | `int64` |
| `day.condition.text` | `object` |
| `day.condition.code` | `int64` |
| `day.uv` | `float64` |
| `astro.sunrise` | `object` |
| `astro.sunset` | `object` |


**2. Are there columns with inappropriate data types?**

Có, dựa trên việc kiểm tra sơ bộ, nhiều cột cần được chuẩn hóa để phù hợp với việc phân tích và tối ưu hóa hiệu suất hệ thống:

* **Dữ liệu Chuỗi thời gian (`date`):** Hiện đang ở dạng chuỗi ký tự (`object`), không thể thực hiện các phép toán thời gian (như lấy tháng, năm, thứ).
* **Dữ liệu Giờ giấc (`astro.sunrise`, `astro.sunset`):** Đang là chuỗi văn bản chỉ chứa giờ (ví dụ "05:30 AM"). Chúng thiếu thông tin ngày tháng để trở thành một mốc thời gian (timestamp) hợp lệ.
* **Biến Phân loại (Categorical):** Các cột `location.name`, `location.region`, `location.terrain`, và `day.condition.text` chứa dữ liệu văn bản lặp lại nhiều lần. Lưu trữ dưới dạng `object` gây tốn bộ nhớ không cần thiết.
* **Biến Logic (`day.daily_will_it_rain`):** Đang lưu trữ dưới dạng số nguyên (`int64`: 0 và 1), trong khi bản chất của dữ liệu là nhị phân (Đúng/Sai).

**3. Which columns need type conversion?**

Dựa trên các vấn đề đã xác định, chúng tôi thực hiện quy trình chuyển đổi sau:

| Tên Cột | Kiểu hiện tại | Kiểu mục tiêu (Target) | Phương pháp & Lý do chuyển đổi |
| :--- | :--- | :--- | :--- |
| `date` | `object` | **`datetime64[ns]`** | Chuyển đổi để trích xuất các đặc trưng thời gian (Tháng, Năm) và lập chỉ mục (indexing). |
| `astro.sunrise` | `object` | **`datetime64[ns]`** | **Ghép với cột `date`:** Tạo thành mốc thời gian đầy đủ (Ngày + Giờ) để tính toán chính xác. |
| `astro.sunset` | `object` | **`datetime64[ns]`** | **Ghép với cột `date`:** Tạo thành mốc thời gian đầy đủ (Ngày + Giờ). |
| `location.name` | `object` | **`category`** | Tối ưu hóa bộ nhớ do các tên tỉnh/thành phố lặp lại thường xuyên. |
| `location.region` | `object` | **`category`** | Tối ưu hóa hiệu suất cho biến phân loại vùng miền. |
| `location.terrain`| `object` | **`category`** | Tối ưu hóa hiệu suất cho biến phân loại địa hình. |
| `day.condition.text`| `object` | **`category`** | Tối ưu hóa bộ nhớ vì các mô tả thời tiết (ví dụ: "Sunny") lặp lại nhiều. |
| `day.daily_will_it_rain`| `int64` | **`bool`** | Chuyển đổi 0/1 thành True/False để phản ánh đúng bản chất Boolean. |